# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [2]:
df = pd.read_csv('/dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [3]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [4]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [5]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location


In [6]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [7]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [8]:
import datetime
import pathway as pw


data = data.with_columns(
    day=pw.apply(lambda x: str(x).split()[0], pw.this.Timestamp)  # Use your actual column name here
)


delta_window = (
    data.groupby(pw.this.day)
    .reduce(
        day=pw.this.day,
        window_end=pw.reducers.max(pw.this.Timestamp),
        Occupancy=pw.reducers.argmax(pw.this.Timestamp, pw.this.Occupancy),
        Capacity=pw.reducers.argmax(pw.this.Timestamp,pw.this.Capacity)
    )
)

# If you need the time window approach, try this syntax:
# delta_window = (
#     data
#     .windowby(
#         pw.this.Timestamp,
#         window=pw.temporal.tumbling(datetime.timedelta(days=1)),
#         instance=pw.this.day
#     )
#     .reduce(
#         window_end=pw.reducers.max(pw.this.Timestamp),
#         Occupancy=pw.reducers.last(pw.this.accuracy),
#         Capacity=pw.reducers.last(pw.this.capacity)
#     )
# )


delta_window = delta_window.with_columns(
    raw_price=10 + 0.5 * (pw.this.Occupancy / pw.this.Capacity)
)


delta_window = delta_window.with_columns(
    price=pw.if_else(
        pw.this.raw_price < 5,
        5,
        pw.if_else(pw.this.raw_price > 20, 20, pw.this.raw_price)
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [9]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
    )
    fig.line("window_end", "price", source=source, line_width=2, color="navy")
    fig.circle("window_end", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="window_end")

pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [11]:
%%capture --no-display
pw.run()

Output()

In [24]:
import pathway as pw
import numpy as np

# Model 2: Demand-Based Price Function (Pathway Type-Safe Version)
def create_demand_based_model():
    """
    Implement Model 2: Demand-Based Price Function with proper Pathway typing
    Uses only native Pathway operations to avoid type inference issues
    """

    # Define schema for your CSV file
    class InputSchema(pw.Schema):
        timestamp: str
        location_id: str
        occupancy_rate: float
        capacity: int
        queue_length: int
        traffic_level: float
        is_special_day: int
        vehicle_type: str
        base_price: float

    # Read streaming data with proper schema
    data_stream = pw.io.csv.read(
        "dataset.csv",
        schema=InputSchema,
        mode="streaming"
    )

    # Create vehicle type mapping table
    vehicle_weights_data = [
        {"vehicle_type": "sedan", "weight": 1.0},
        {"vehicle_type": "suv", "weight": 1.2},
        {"vehicle_type": "truck", "weight": 1.5},
        {"vehicle_type": "motorcycle", "weight": 0.8}
    ]

    class VehicleWeightSchema(pw.Schema):
        vehicle_type: str
        weight: float

    # Create static table for vehicle weights
    vehicle_weights = pw.debug.table_from_rows(
        schema=VehicleWeightSchema,
        rows=vehicle_weights_data
    )

    # Step 1: Join with vehicle weights to avoid pw.apply
    step1 = data_stream.join(
        vehicle_weights,
        data_stream.vehicle_type == vehicle_weights.vehicle_type,
        how="left"
    ).select(
        timestamp=data_stream.timestamp,
        location_id=data_stream.location_id,
        occupancy_rate=data_stream.occupancy_rate,
        capacity=data_stream.capacity,
        queue_length=data_stream.queue_length,
        traffic_level=data_stream.traffic_level,
        is_special_day=data_stream.is_special_day,
        vehicle_type=data_stream.vehicle_type,
        base_price=data_stream.base_price,
        vehicle_weight=pw.coalesce(vehicle_weights.weight, 1.0)  # Default weight if not found
    )

    # Step 2: Calculate occupancy ratio
    step2 = step1.with_columns(
        occupancy_ratio=pw.this.occupancy_rate / pw.cast(float, pw.this.capacity),
        queue_normalized=pw.cast(float, pw.this.queue_length) / 10.0,  # Normalize queue (assume max 10)
        special_day_float=pw.cast(float, pw.this.is_special_day)
    )

    # Step 3: Calculate demand components using only native operations
    step3 = step2.with_columns(
        # Base demand from occupancy (weight: 0.5)
        occupancy_demand=pw.this.occupancy_ratio * 0.5,

        # Queue demand (weight: 0.2, capped at 0.2)
        queue_demand=pw.if_else(
            pw.this.queue_normalized > 1.0,
            0.2,
            pw.this.queue_normalized * 0.2
        ),

        # Special day bonus (weight: 0.2)
        special_demand=pw.this.special_day_float * 0.2,

        # Traffic penalty (weight: -0.1)
        traffic_penalty=pw.this.traffic_level * 0.1
    )

    # Step 4: Combine demand components
    step4 = step3.with_columns(
        raw_demand=pw.this.occupancy_demand + pw.this.queue_demand + pw.this.special_demand - pw.this.traffic_penalty
    )

    # Step 5: Normalize demand to [0, 1] range
    step5 = step4.with_columns(
        normalized_demand=pw.if_else(
            pw.this.raw_demand > 1.0,
            1.0,
            pw.if_else(
                pw.this.raw_demand < 0.0,
                0.0,
                pw.this.raw_demand
            )
        )
    )

    # Step 6: Calculate price multiplier (1.0 + 0.5 * demand)
    step6 = step5.with_columns(
        price_multiplier=1.0 + (pw.this.normalized_demand * 0.5)
    )

    # Step 7: Calculate predicted price with vehicle weight
    step7 = step6.with_columns(
        predicted_price=pw.this.base_price * pw.this.price_multiplier * pw.this.vehicle_weight
    )

    # Step 8: Apply price bounds (50% to 150% of base price)
    step8 = step7.with_columns(
        min_price=pw.this.base_price * 0.5,
        max_price=pw.this.base_price * 1.5
    ).with_columns(
        bounded_price=pw.if_else(
            pw.this.predicted_price > pw.this.max_price,
            pw.this.max_price,
            pw.if_else(
                pw.this.predicted_price < pw.this.min_price,
                pw.this.min_price,
                pw.this.predicted_price
            )
        )
    )

    # Step 9: Calculate price adjustment percentage
    final_pricing = step8.with_columns(
        price_adjustment=((pw.this.bounded_price - pw.this.base_price) / pw.this.base_price) * 100.0
    )

    # Select final columns for output
    demand_based_pricing = final_pricing.select(
        timestamp=pw.this.timestamp,
        location_id=pw.this.location_id,
        occupancy_rate=pw.this.occupancy_rate,
        capacity=pw.this.capacity,
        queue_length=pw.this.queue_length,
        traffic_level=pw.this.traffic_level,
        is_special_day=pw.this.is_special_day,
        vehicle_type=pw.this.vehicle_type,
        base_price=pw.this.base_price,
        vehicle_weight=pw.this.vehicle_weight,
        normalized_demand=pw.this.normalized_demand,
        predicted_price=pw.this.predicted_price,
        bounded_price=pw.this.bounded_price,
        price_adjustment=pw.this.price_adjustment
    )

    # Add real-time aggregations for monitoring
    demand_summary = demand_based_pricing.groupby(pw.this.location_id).reduce(
        location_id=pw.this.location_id,
        avg_demand=pw.reducers.avg(pw.this.normalized_demand),
        avg_price=pw.reducers.avg(pw.this.bounded_price),
        max_price=pw.reducers.max(pw.this.bounded_price),
        min_price=pw.reducers.min(pw.this.bounded_price),
        total_transactions=pw.reducers.count()
    )

    return demand_based_pricing, demand_summary

# Alternative: Ultra-simple version with hardcoded vehicle weights
def create_ultra_simple_model():
    """
    Ultra-simplified version that avoids joins and uses hardcoded logic
    """

    class InputSchema(pw.Schema):
        timestamp: str
        location_id: str
        occupancy_rate: float
        capacity: int
        queue_length: int
        traffic_level: float
        is_special_day: int
        vehicle_type: str
        base_price: float

    # Read streaming data
    data_stream = pw.io.csv.read(
        "dataset.csv",
        schema=InputSchema,
        mode="streaming"
    )

    # Process with hardcoded vehicle weights using case expressions
    processed_data = data_stream.with_columns(
        # Hardcode vehicle weights using case expressions
        vehicle_weight=pw.if_else(
            pw.this.vehicle_type == "suv",
            1.2,
            pw.if_else(
                pw.this.vehicle_type == "truck",
                1.5,
                pw.if_else(
                    pw.this.vehicle_type == "motorcycle",
                    0.8,
                    1.0  # Default for sedan and others
                )
            )
        )
    ).with_columns(
        # Calculate basic metrics
        occupancy_ratio=pw.this.occupancy_rate / pw.cast(float, pw.this.capacity),
        queue_factor=pw.cast(float, pw.this.queue_length) * 0.05,  # Simple queue factor
        special_bonus=pw.cast(float, pw.this.is_special_day) * 0.3,
        traffic_penalty=pw.this.traffic_level * 0.2
    ).with_columns(
        # Calculate demand score
        raw_demand=pw.this.occupancy_ratio + pw.this.queue_factor + pw.this.special_bonus - pw.this.traffic_penalty
    ).with_columns(
        # Clamp demand to [0, 1]
        demand_score=pw.if_else(
            pw.this.raw_demand > 1.0,
            1.0,
            pw.if_else(pw.this.raw_demand < 0.0, 0.0, pw.this.raw_demand)
        )
    ).with_columns(
        # Calculate price with demand and vehicle factors
        price_factor=1.0 + (pw.this.demand_score * 0.5),
        adjusted_price=pw.this.base_price * (1.0 + (pw.this.demand_score * 0.5)) * pw.this.vehicle_weight
    ).with_columns(
        # Apply bounds
        final_price=pw.if_else(
            pw.this.adjusted_price > (pw.this.base_price * 1.5),
            pw.this.base_price * 1.5,
            pw.if_else(
                pw.this.adjusted_price < (pw.this.base_price * 0.5),
                pw.this.base_price * 0.5,
                pw.this.adjusted_price
            )
        )
    ).with_columns(
        # Calculate adjustment percentage
        price_adjustment=((pw.this.final_price - pw.this.base_price) / pw.this.base_price) * 100.0
    )

    # Select output columns
    result = processed_data.select(
        timestamp=pw.this.timestamp,
        location_id=pw.this.location_id,
        occupancy_rate=pw.this.occupancy_rate,
        capacity=pw.this.capacity,
        queue_length=pw.this.queue_length,
        traffic_level=pw.this.traffic_level,
        is_special_day=pw.this.is_special_day,
        vehicle_type=pw.this.vehicle_type,
        base_price=pw.this.base_price,
        demand_score=pw.this.demand_score,
        final_price=pw.this.final_price,
        price_adjustment=pw.this.price_adjustment
    )

    return result

# Implementation and execution
def run_model_2():
    """
    Execute Model 2: Demand-Based Price Function
    """
    print("Starting Model 2: Demand-Based Price Function (Type-Safe)")

    try:
        # Try the main model first
        print("Attempting main model with joins...")
        pricing_results, summary_stats = create_demand_based_model()

        # Output detailed pricing results
        pw.io.csv.write(pricing_results, "model2_demand_pricing_results.csv")

        # Output summary statistics
        pw.io.csv.write(summary_stats, "model2_demand_summary.csv")

        # For real-time monitoring
        pw.io.subscribe(
            pricing_results,
            on_change=lambda key, row, time, is_addition:
                print(f"Price Update: Location {row['location_id']}, "
                      f"Demand: {row['normalized_demand']:.3f}, "
                      f"Price: ${row['bounded_price']:.2f}, "
                      f"Adjustment: {row['price_adjustment']:.1f}%")
        )

        print("Model 2 setup complete. Running real-time processing...")

        # Run the model
        pw.run()

    except Exception as e:
        print(f"Error in main model: {str(e)}")
        print("Trying ultra-simple version...")

        try:
            # Fallback to ultra-simple model
            pricing_results_simple = create_ultra_simple_model()

            pw.io.csv.write(pricing_results_simple, "model2_ultra_simple_results.csv")

            # Simple monitoring
            pw.io.subscribe(
                pricing_results_simple,
                on_change=lambda key, row, time, is_addition:
                    print(f"Price Update: Location {row['location_id']}, "
                          f"Demand: {row['demand_score']:.3f}, "
                          f"Price: ${row['final_price']:.2f}")
            )

            print("Ultra-simple model setup complete. Running...")
            pw.run()

        except Exception as e2:
            print(f"Error in ultra-simple model: {str(e2)}")
            print("Pathway type system is very restrictive. Here's what we tried:")
            print("1. Using pw.apply() - fails due to ANY type inference")
            print("2. Using joins for lookups - may fail on table creation")
            print("3. Using only native operations - still may have type issues")
            print("\nFor Pathway to work, you might need to:")
            print("- Ensure your CSV has proper headers and data types")
            print("- Use a simpler approach with fewer calculations")
            print("- Check Pathway documentation for the specific version you're using")
            raise

# Simple debug version for testing
def create_debug_model():
    """
    Minimal version for debugging Pathway setup
    """
    class InputSchema(pw.Schema):
        timestamp: str
        location_id: str
        occupancy_rate: float
        capacity: int
        base_price: float

    data_stream = pw.io.csv.read(
        "dataset.csv",
        schema=InputSchema,
        mode="streaming"
    )

    # Minimal processing
    result = data_stream.with_columns(
        occupancy_ratio=pw.this.occupancy_rate / pw.cast(float, pw.this.capacity),
        adjusted_price=pw.this.base_price * 1.1  # Simple 10% increase
    )

    return result

def run_debug_model():
    """Test minimal Pathway functionality"""
    print("Running debug model...")
    try:
        result = create_debug_model()
        pw.io.csv.write(result, "debug_results.csv")
        pw.run()
        print("Debug model completed successfully!")
    except Exception as e:
        print(f"Even debug model failed: {str(e)}")
        print("This suggests a fundamental issue with your Pathway setup or data file.")

if __name__ == "__main__":
    # Try main model, fall back to debug if needed
    try:
        run_model_2()
    except Exception:
        print("\n" + "="*50)
        print("All models failed. Trying minimal debug model...")
        run_debug_model()

Starting Model 2: Demand-Based Price Function (Type-Safe)
Attempting main model with joins...
Error in main model: Function pathway.debug.table_from_rows() parameter rows=[{'vehicle_type': 'sedan', 'weight': 1.0}, {'vehicle_type': 'suv', 'weight': 1.2}, {'vehicle_... violates type hint list[tuple], as list index 0 item dict {'vehicle_type': 'sedan', 'weight': 1.0} not instance of tuple.
Trying ultra-simple version...
Ultra-simple model setup complete. Running...


Output()

Error in ultra-simple model: Pathway doesn't support casting ANY to FLOAT.
Pathway type system is very restrictive. Here's what we tried:
1. Using pw.apply() - fails due to ANY type inference
2. Using joins for lookups - may fail on table creation
3. Using only native operations - still may have type issues

For Pathway to work, you might need to:
- Ensure your CSV has proper headers and data types
- Use a simpler approach with fewer calculations
- Check Pathway documentation for the specific version you're using

All models failed. Trying minimal debug model...
Running debug model...


Output()

Even debug model failed: Pathway doesn't support casting ANY to FLOAT.
This suggests a fundamental issue with your Pathway setup or data file.
